In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from radical.entk import Profiler
import radical.analytics as ra
import radical.utils as ru
import radical.pilot as rp

%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
# Generate task uids

def get_task_uids(num_pipelines):
    
    num_tasks = num_pipelines*7*1
    task_uids = []
    for t in range(num_tasks):
        task_uids.append('radical.entk.task.%04d'%t)

    return task_uids

In [3]:
# Get information from all the entk profiles

def get_entk_info(src):
    
    p = Profiler(src = src + '/../')
    
    task_uids = get_task_uids(pipelines)
    
    entk_total_task_dur = p.duration(task_uids, states=['SCHEDULING','DONE'])
    entk_task_exec_dur = p.duration(task_uids, states=['SUBMITTED','EXECUTED'])
    
    print 'EnTK total task duration: ', entk_total_task_dur
    print 'EnTK task execution duration: ', entk_task_exec_dur
    

In [4]:
# Get information from all the rp profile files and json file
# returns 0,0 if no rp files are found

def get_rp_info(src):

    json_files = glob.glob('%s/*.json'%src)
    json_file = json_files[0]
    json      = ru.read_json(json_file)
    sid       = os.path.basename(json_file)[:-5]

    session = ra.Session(sid, 'radical.pilot', src=src)
    units = session.filter(etype='unit', inplace=False)

    exec_dur = units.duration([rp.AGENT_EXECUTING, rp.AGENT_STAGING_OUTPUT_PENDING])
    rp_dur = units.duration([rp.UMGR_SCHEDULING, rp.DONE])
    
#     for unit in units.get()[:1]:
#         print unit

    print 'RP TTE: ', exec_dur
    
    # Note: This is the TTR derived from the units, not the pilot. All tasks are submitted after the pilot is Active
    print 'RP TTR: ', rp_dur


In [7]:
df = pd.DataFrame(columns=['EnTK overhead', 'RP overhead', 'Execution duration'])

pipelines_list = [8]

for pipelines in pipelines_list:
    
    src = glob.glob('./rp.session*'.format(pipelines))
    src_cpy = list(src)
    #print src
    for f in src:
        if f.split('.')[-1] == 'json':
            src_cpy.remove(f)

    src = src_cpy[0]
    
    get_entk_info(src)
    get_rp_info(src)    

EnTK total task duration:  2640.26300001
EnTK task execution duration:  2630.51989985
RP TTE:  2586.2560997
RP TTR:  2561.21889997
